## Training the model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
#  import the dataset
X, y = load_iris(return_X_y=True, as_frame=True)
# split in training and test set
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
logreg = LogisticRegression()
# Create an instance of Logistic Regression Classifier and train the model.
logreg.fit(X_train, y_train)

### Save and load model

In [ ]:
import joblib
path = 'log_model.joblib'
joblib.dump(model, path)

## load model
log_model = joblib.load(path)

## WRAP MODEL INTO SPARK UDF

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import udf
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
broadcast_model = sc.broadcast(log_model)
@udf('integer')
def predict_data(*cols):
   return int(broadcast_model.value.predict((cols,)))

## Read the data and make predictions

In [ ]:
# sample of unseen data
df = spark.createDataFrame(X_test)

# get columns and call the predict_data function
list_of_columns = df.columns
df = df.withColumn("prediction", predict_data(*list_of_columns))